# 02 - Preprocessing Dataset Industrial IoT

Questo notebook preprocessa il dataset dei dispositivi IoT industriali:
- Pulizia dei dati e gestione valori mancanti
- Feature engineering e creazione nuove variabili
- Normalizzazione e standardizzazione
- Gestione outliers
- Divisione train/test set
- Salvataggio dati processati

In [ ]:
# Import delle librerie necessarie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import yaml
import joblib
import warnings
import os

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurazione per i grafici
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Caricamento configurazione e dataset
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Caricamento dataset originale
data_path = '../' + config['data']['raw_data_path']
df = pd.read_csv(data_path)

print(f"Dataset caricato: {df.shape}")
print(f"Memoria utilizzata: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 1. Pulizia Iniziale dei Dati

In [ ]:
# Controllo duplicati
print("=== CONTROLLO DUPLICATI ===")
duplicates = df.duplicated().sum()
print(f"Righe duplicate: {duplicates}")

if duplicates > 0:
    print("Rimozione duplicati...")
    df = df.drop_duplicates()
    print(f"Nuova forma dataset: {df.shape}")

# Reset dell'indice
df = df.reset_index(drop=True)

# Controllo valori mancanti per colonna
print("\n=== VALORI MANCANTI ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Colonna': missing_data.index,
    'Valori_Mancanti': missing_data.values,
    'Percentuale': missing_percent.values
})

missing_df = missing_df[missing_df['Valori_Mancanti'] > 0].sort_values('Percentuale', ascending=False)
if len(missing_df) > 0:
    display(missing_df)
else:
    print("✅ Nessun valore mancante trovato!")

In [ ]:
# Classificazione dispositivi secondo le specifiche del progetto
devices_common_only = config['devices_common_only']
laser_devices = config['devices_with_additional']['laser_devices']
hydraulic_devices = config['devices_with_additional']['hydraulic_devices']
coolant_devices = config['devices_with_additional']['coolant_devices']
heat_devices = config['devices_with_additional']['heat_devices']

def classify_device(machine_type):
    if machine_type in devices_common_only:
        return 'Solo Comuni'
    elif machine_type in laser_devices:
        return 'Laser'
    elif machine_type in hydraulic_devices:
        return 'Idraulico'
    elif machine_type in coolant_devices:
        return 'Refrigerante'
    elif machine_type in heat_devices:
        return 'Calore'
    else:
        return 'Altro'

df['Device_Category'] = df['Machine_Type'].apply(classify_device)
print("✅ Classificazione dispositivi completata")

## 2. Gestione Valori Mancanti

In [ ]:
# Gestione valori mancanti per features aggiuntive
print("=== GESTIONE VALORI MANCANTI FEATURES AGGIUNTIVE ===")

# Le features aggiuntive devono essere nulle per i dispositivi che non le utilizzano
additional_features = config['features']['additional_features']

# Laser_Intensity: solo per dispositivi laser
df.loc[~df['Machine_Type'].isin(laser_devices), 'Laser_Intensity'] = np.nan

# Hydraulic_Pressure_bar: solo per dispositivi idraulici
df.loc[~df['Machine_Type'].isin(hydraulic_devices), 'Hydraulic_Pressure_bar'] = np.nan

# Coolant_Flow_L_min: solo per dispositivi con refrigerante
df.loc[~df['Machine_Type'].isin(coolant_devices), 'Coolant_Flow_L_min'] = np.nan

# Heat_Index: solo per dispositivi con calore
df.loc[~df['Machine_Type'].isin(heat_devices), 'Heat_Index'] = np.nan

# Per i dispositivi che dovrebbero avere queste features, imputa i valori mancanti
# Usando la mediana per ogni gruppo di dispositivi

for device_type in laser_devices:
    mask = df['Machine_Type'] == device_type
    if mask.sum() > 0 and 'Laser_Intensity' in df.columns:
        median_val = df.loc[mask, 'Laser_Intensity'].median()
        df.loc[mask & df['Laser_Intensity'].isna(), 'Laser_Intensity'] = median_val

for device_type in hydraulic_devices:
    mask = df['Machine_Type'] == device_type
    if mask.sum() > 0 and 'Hydraulic_Pressure_bar' in df.columns:
        median_val = df.loc[mask, 'Hydraulic_Pressure_bar'].median()
        df.loc[mask & df['Hydraulic_Pressure_bar'].isna(), 'Hydraulic_Pressure_bar'] = median_val

for device_type in coolant_devices:
    mask = df['Machine_Type'] == device_type
    if mask.sum() > 0 and 'Coolant_Flow_L_min' in df.columns:
        median_val = df.loc[mask, 'Coolant_Flow_L_min'].median()
        df.loc[mask & df['Coolant_Flow_L_min'].isna(), 'Coolant_Flow_L_min'] = median_val

for device_type in heat_devices:
    mask = df['Machine_Type'] == device_type
    if mask.sum() > 0 and 'Heat_Index' in df.columns:
        median_val = df.loc[mask, 'Heat_Index'].median()
        df.loc[mask & df['Heat_Index'].isna(), 'Heat_Index'] = median_val

print("✅ Gestione valori mancanti completata")

# Controllo finale valori mancanti
final_missing = df.isnull().sum()
if final_missing.sum() > 0:
    print("\nValori mancanti rimanenti:")
    print(final_missing[final_missing > 0])
else:
    print("✅ Nessun valore mancante rimanente!")

In [ ]:
# Verifica conformità delle 4 colonne speciali secondo le specifiche del progetto
print("=== VERIFICA CONFORMITÀ 4 COLONNE SPECIALI ===")

special_columns_check = {
    'Laser_Intensity': {
        'should_have': laser_devices,
        'should_not_have': [d for d in df['Machine_Type'].unique() if d not in laser_devices]
    },
    'Hydraulic_Pressure_bar': {
        'should_have': hydraulic_devices,
        'should_not_have': [d for d in df['Machine_Type'].unique() if d not in hydraulic_devices]
    },
    'Coolant_Flow_L_min': {
        'should_have': coolant_devices,
        'should_not_have': [d for d in df['Machine_Type'].unique() if d not in coolant_devices]
    },
    'Heat_Index': {
        'should_have': heat_devices,
        'should_not_have': [d for d in df['Machine_Type'].unique() if d not in heat_devices]
    }
}

conformity_results = {}
for col, devices in special_columns_check.items():
    if col in df.columns:
        # Verifica che dispositivi che non dovrebbero avere la colonna abbiano NaN
        wrong_values = df[df['Machine_Type'].isin(devices['should_not_have']) & 
                         df[col].notna()].shape[0]
        
        # Verifica percentuale valori mancanti per dispositivi che dovrebbero averla
        should_have_count = df[df['Machine_Type'].isin(devices['should_have'])].shape[0]
        missing_count = df[df['Machine_Type'].isin(devices['should_have']) & 
                          df[col].isna()].shape[0]
        missing_pct = (missing_count / should_have_count * 100) if should_have_count > 0 else 0
        
        conformity_results[col] = {
            'wrong_values': wrong_values,
            'missing_percentage': missing_pct
        }
        
        if wrong_values > 0:
            print(f"⚠️ {col}: {wrong_values} dispositivi hanno valori quando dovrebbero essere NaN")
        else:
            print(f"✅ {col}: Correttamente NULL per dispositivi non applicabili")
        
        if should_have_count > 0:
            print(f"   {col}: {missing_pct:.1f}% valori mancanti per dispositivi che dovrebbero averla")

print("\n✅ Verifica conformità 4 colonne speciali completata")

## 3. Feature Engineering

In [ ]:
# Creazione nuove features derivate
print("=== FEATURE ENGINEERING ===")

# 1. Età del dispositivo
current_year = 2024
df['Device_Age_Years'] = current_year - df['Installation_Year']

# 2. Intensità di utilizzo
df['Usage_Intensity'] = df['Operational_Hours'] / (df['Device_Age_Years'] * 365 * 24)
df['Usage_Intensity'] = df['Usage_Intensity'].fillna(0)  # Per dispositivi installati quest'anno

# 3. Rapporto manutenzioni/età
df['Maintenance_Rate'] = df['Maintenance_History_Count'] / (df['Device_Age_Years'] + 1)  # +1 per evitare divisione per 0

# 4. Rapporto guasti/età
df['Failure_Rate'] = df['Failure_History_Count'] / (df['Device_Age_Years'] + 1)

# 5. Indicatore manutenzione recente
df['Recent_Maintenance'] = (df['Last_Maintenance_Days_Ago'] <= 30).astype(int)

# 6. Indicatore alta temperatura
df['High_Temperature'] = (df['Temperature_C'] > df['Temperature_C'].quantile(0.75)).astype(int)

# 7. Indicatore alta vibrazione
df['High_Vibration'] = (df['Vibration_mms'] > df['Vibration_mms'].quantile(0.75)).astype(int)

# 8. Score di salute generale (combinazione di fattori)
# Normalizzazione tra 0 e 1 per ogni componente
temp_norm = (df['Temperature_C'] - df['Temperature_C'].min()) / (df['Temperature_C'].max() - df['Temperature_C'].min())
vibration_norm = (df['Vibration_mms'] - df['Vibration_mms'].min()) / (df['Vibration_mms'].max() - df['Vibration_mms'].min())
oil_norm = df['Oil_Level_pct'] / 100
coolant_norm = df['Coolant_Level_pct'] / 100

# Health score (più alto = più sano)
df['Health_Score'] = ((1 - temp_norm) + (1 - vibration_norm) + oil_norm + coolant_norm) / 4

# 9. Binning età dispositivo
df['Age_Category'] = pd.cut(df['Device_Age_Years'], 
                           bins=[0, 2, 5, 10, float('inf')], 
                           labels=['Nuovo', 'Giovane', 'Maturo', 'Vecchio'])

# 10. Interaction features
df['Temp_Vibration_Interaction'] = df['Temperature_C'] * df['Vibration_mms']
df['Age_Usage_Interaction'] = df['Device_Age_Years'] * df['Usage_Intensity']

print(f"✅ Feature engineering completato. Nuove features create: {len(['Device_Age_Years', 'Usage_Intensity', 'Maintenance_Rate', 'Failure_Rate', 'Recent_Maintenance', 'High_Temperature', 'High_Vibration', 'Health_Score', 'Age_Category', 'Temp_Vibration_Interaction', 'Age_Usage_Interaction'])}")
print(f"Forma dataset aggiornata: {df.shape}")

## 4. Gestione Outliers

In [ ]:
# Identificazione e gestione outliers usando IQR
def handle_outliers(data, column, method='cap'):
    """
    Gestisce gli outliers usando il metodo IQR
    method: 'cap' per cappare, 'remove' per rimuovere
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_count = len(data[(data[column] < lower_bound) | (data[column] > upper_bound)])
    
    if method == 'cap':
        data[column] = data[column].clip(lower_bound, upper_bound)
    elif method == 'remove':
        data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    
    return data, outliers_count

print("=== GESTIONE OUTLIERS ===")

# Features numeriche da processare (escludendo target e ID)
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
features_to_process = [f for f in numeric_features if f not in ['Machine_ID', 'Failure_Within_7_Days', 'Remaining_Useful_Life_days']]

outlier_summary = []

for feature in features_to_process[:15]:  # Primi 15 per non sovraccaricare
    df_processed, outliers_count = handle_outliers(df.copy(), feature, method='cap')
    if outliers_count > 0:
        df[feature] = df_processed[feature]
        outlier_summary.append({
            'Feature': feature,
            'Outliers_Rimossi': outliers_count,
            'Percentuale': (outliers_count / len(df)) * 100
        })

if outlier_summary:
    outlier_df = pd.DataFrame(outlier_summary).sort_values('Percentuale', ascending=False)
    display(outlier_df)
    print(f"\n✅ Outliers gestiti per {len(outlier_summary)} features")
else:
    print("✅ Nessun outlier significativo trovato")

## 5. Encoding Variabili Categoriche

In [ ]:
# Encoding delle variabili categoriche
print("=== ENCODING VARIABILI CATEGORICHE ===")

# Label Encoder per Machine_Type
le_machine = LabelEncoder()
df['Machine_Type_Encoded'] = le_machine.fit_transform(df['Machine_Type'])

# Label Encoder per Device_Category
le_category = LabelEncoder()
df['Device_Category_Encoded'] = le_category.fit_transform(df['Device_Category'])

# One-hot encoding per Age_Category
age_dummies = pd.get_dummies(df['Age_Category'], prefix='Age')
df = pd.concat([df, age_dummies], axis=1)

print(f"✅ Encoding completato")
print(f"Machine types unici: {df['Machine_Type'].nunique()}")
print(f"Device categories: {df['Device_Category'].nunique()}")
print(f"Nuove colonne age: {list(age_dummies.columns)}")

# Salvataggio encoders
os.makedirs('../data/models', exist_ok=True)
joblib.dump(le_machine, '../data/models/machine_type_encoder.pkl')
joblib.dump(le_category, '../data/models/device_category_encoder.pkl')
print("✅ Encoders salvati")

## 6. Selezione Features Finali

In [ ]:
# Definizione features finali per il modelling
print("=== SELEZIONE FEATURES FINALI ===")

# Features comuni
common_features = config['features']['common_features']

# Features aggiuntive (sostituiamo NaN con 0 per modelling)
additional_features = config['features']['additional_features']

# Features engineered
engineered_features = [
    'Device_Age_Years', 'Usage_Intensity', 'Maintenance_Rate', 'Failure_Rate',
    'Recent_Maintenance', 'High_Temperature', 'High_Vibration', 'Health_Score',
    'Temp_Vibration_Interaction', 'Age_Usage_Interaction'
]

# Features encoded
encoded_features = ['Machine_Type_Encoded', 'Device_Category_Encoded']

# Age category dummies
age_features = [col for col in df.columns if col.startswith('Age_')]

# Combina tutte le features
all_features = common_features + additional_features + engineered_features + encoded_features + age_features

# Filtra solo le features che esistono nel dataframe
available_features = [f for f in all_features if f in df.columns]

print(f"Features disponibili per il modelling: {len(available_features)}")
print("Features selezionate:")
for i, feature in enumerate(available_features, 1):
    print(f"  {i:2d}. {feature}")

# Riempimento NaN nelle additional features con 0 (per dispositivi che non le hanno)
for feature in additional_features:
    if feature in df.columns:
        df[feature] = df[feature].fillna(0)

# Verifica finale NaN
final_missing = df[available_features].isnull().sum().sum()
print(f"\nValori mancanti nelle features selezionate: {final_missing}")

## 7. Standardizzazione Features

In [ ]:
# Preparazione dati per standardizzazione
print("=== STANDARDIZZAZIONE FEATURES ===")

# Separazione features e target
X = df[available_features].copy()
y_classification = df['Failure_Within_7_Days'].copy()
y_regression = df['Remaining_Useful_Life_days'].copy()

print(f"Forma X: {X.shape}")
print(f"Forma y_classification: {y_classification.shape}")
print(f"Forma y_regression: {y_regression.shape}")

# Identificazione features numeriche per standardizzazione
numeric_features_to_scale = X.select_dtypes(include=[np.number]).columns.tolist()
# Escludiamo le features già binarie/categoriche encoded
features_not_to_scale = ['Recent_Maintenance', 'High_Temperature', 'High_Vibration', 
                        'Machine_Type_Encoded', 'Device_Category_Encoded'] + age_features
features_to_scale = [f for f in numeric_features_to_scale if f not in features_not_to_scale]

print(f"\nFeatures da standardizzare: {len(features_to_scale)}")
print(f"Features non standardizzate: {len(features_not_to_scale)}")

# Standardizzazione
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[features_to_scale] = scaler.fit_transform(X[features_to_scale])

# Salvataggio scaler
joblib.dump(scaler, '../data/models/scaler.pkl')
print("✅ Scaler salvato")

# Verifica standardizzazione
print("\nStatistiche dopo standardizzazione (features scalate):")
print(f"Media: {X_scaled[features_to_scale].mean().mean():.6f}")
print(f"Std: {X_scaled[features_to_scale].std().mean():.6f}")

In [ ]:
# Validazione della qualità delle features dopo standardizzazione
print("=== VALIDAZIONE QUALITÀ FEATURES ===")

# 1. Controllo varianza features standardizzate
print("\n1. Controllo Varianza Features:")
low_variance_features = []
for feature in features_to_scale:
    if feature in X_scaled.columns:
        variance = X_scaled[feature].var()
        if variance < 0.01:
            low_variance_features.append((feature, variance))

if low_variance_features:
    print("⚠️ Features con bassa varianza (potrebbero essere poco informative):")
    for feat, var in low_variance_features:
        print(f"  {feat}: varianza = {var:.6f}")
else:
    print("✅ Tutte le features hanno varianza appropriata")

# 2. Controllo coerenza temporale
print("\n2. Controllo Coerenza Temporale:")

# Verifica ore operative vs età dispositivo
df['theoretical_max_hours'] = df['Device_Age_Years'] * 365 * 24
df['usage_ratio'] = df['Operational_Hours'] / df['theoretical_max_hours']
df['usage_ratio'] = df['usage_ratio'].fillna(0)  # Per dispositivi nuovi

impossible_usage = (df['usage_ratio'] > 1.2).sum()  # Più di 120% utilizzo teorico
very_high_usage = ((df['usage_ratio'] > 0.8) & (df['usage_ratio'] <= 1.2)).sum()

print(f"Dispositivi con utilizzo > 120% teorico: {impossible_usage}")
print(f"Dispositivi con utilizzo 80-120% (molto alto): {very_high_usage}")

if impossible_usage > 0:
    print("⚠️ Alcuni dispositivi hanno ore operative irrealisticamente alte")
    # Mostra alcuni esempi
    extreme_cases = df[df['usage_ratio'] > 1.2][['Machine_Type', 'Device_Age_Years', 
                                                 'Operational_Hours', 'usage_ratio']].head()
    if len(extreme_cases) > 0:
        print("Esempi di casi estremi:")
        display(extreme_cases.round(2))
else:
    print("✅ Ore operative coerenti con età dispositivi")

# Verifica coerenza manutenzioni
maintenance_age_issues = (df['Last_Maintenance_Days_Ago'] > df['Device_Age_Years'] * 365).sum()
if maintenance_age_issues > 0:
    print(f"⚠️ {maintenance_age_issues} dispositivi con ultima manutenzione precedente all'installazione")
else:
    print("✅ Date manutenzione coerenti con età dispositivi")

# 3. Controllo distribuzione features chiave
print("\n3. Distribuzione Features Chiave:")
key_features = ['Temperature_C', 'Vibration_mms', 'Health_Score', 'Usage_Intensity']
for feature in key_features:
    if feature in df.columns:
        q25, q50, q75 = df[feature].quantile([0.25, 0.5, 0.75])
        skewness = df[feature].skew()
        print(f"{feature}: Q25={q25:.2f}, Mediana={q50:.2f}, Q75={q75:.2f}, Skew={skewness:.2f}")
        
        if abs(skewness) > 2:
            print(f"  ⚠️ {feature} ha distribuzione molto asimmetrica (skew={skewness:.2f})")

print("\n✅ Validazione qualità features completata")

## 8. Divisione Train/Test Set

In [ ]:
# Divisione train/test set
print("=== DIVISIONE TRAIN/TEST SET ===")

test_size = config['model']['test_size']
random_state = config['model']['random_state']

# Stratified split basato sul target di classificazione
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X_scaled, y_classification, y_regression,
    test_size=test_size,
    random_state=random_state,
    stratify=y_classification
)

print(f"Dimensioni training set: {X_train.shape}")
print(f"Dimensioni test set: {X_test.shape}")

print(f"\nDistribuzione target classificazione - Train:")
print(y_class_train.value_counts(normalize=True).round(3))
print(f"\nDistribuzione target classificazione - Test:")
print(y_class_test.value_counts(normalize=True).round(3))

print(f"\nTarget regressione - Train stats:")
print(f"  Media: {y_reg_train.mean():.2f}")
print(f"  Std: {y_reg_train.std():.2f}")
print(f"\nTarget regressione - Test stats:")
print(f"  Media: {y_reg_test.mean():.2f}")
print(f"  Std: {y_reg_test.std():.2f}")

In [ ]:
# Analisi approfondita del bilanciamento delle classi
print("=== ANALISI BILANCIAMENTO CLASSI ===")

class_distribution = y_classification.value_counts()
total_samples = class_distribution.sum()
minority_class_count = class_distribution.min()
majority_class_count = class_distribution.max()
minority_class_pct = (minority_class_count / total_samples) * 100
imbalance_ratio = majority_class_count / minority_class_count

print(f"Classe maggioritaria: {majority_class_count} campioni ({100-minority_class_pct:.1f}%)")
print(f"Classe minoritaria: {minority_class_count} campioni ({minority_class_pct:.1f}%)")
print(f"Rapporto di sbilanciamento: {imbalance_ratio:.2f}:1")

# Classificazione del livello di sbilanciamento
if minority_class_pct < 5:
    balance_status = "MOLTO SBILANCIATO"
    recommendation = "Utilizzare SMOTE, Random Oversampling o Class Weights nei modelli"
    print(f"🔴 Status: {balance_status}")
elif minority_class_pct < 20:
    balance_status = "MODERATAMENTE SBILANCIATO"
    recommendation = "Considerare Class Weights o strategie di campionamento"
    print(f"🟡 Status: {balance_status}")
else:
    balance_status = "RAGIONEVOLMENTE BILANCIATO"
    recommendation = "Bilanciamento accettabile per il training"
    print(f"🟢 Status: {balance_status}")

print(f"💡 Raccomandazione: {recommendation}")

# Controllo bilanciamento anche nei set train/test
print("\n=== BILANCIAMENTO TRAIN/TEST ===")
train_balance = y_class_train.value_counts(normalize=True).round(3)
test_balance = y_class_test.value_counts(normalize=True).round(3)

print("Distribuzione Train:")
for class_val, pct in train_balance.items():
    print(f"  Classe {class_val}: {pct:.1%}")

print("Distribuzione Test:")
for class_val, pct in test_balance.items():
    print(f"  Classe {class_val}: {pct:.1%}")

# Verifica che la stratificazione abbia funzionato
balance_difference = abs(train_balance[1] - test_balance[1])
if balance_difference < 0.02:
    print("✅ Stratificazione riuscita: distribuzioni train/test molto simili")
else:
    print(f"⚠️ Differenza distribuzione train/test: {balance_difference:.1%}")

## 9. Salvataggio Dati Processati

In [ ]:
# Salvataggio dati processati
print("=== SALVATAGGIO DATI PROCESSATI ===")

# Creazione directory se non esiste
processed_dir = '../data/processed/'
os.makedirs(processed_dir, exist_ok=True)

# Salvataggio dataset completo pulito
df_clean = df.copy()
df_clean.to_csv(processed_dir + 'cleaned_data.csv', index=False)
print(f"✅ Dataset pulito salvato: {df_clean.shape}")

# Preparazione dati per salvataggio train/test
# Combinazione features con target per i file di training
train_data = X_train.copy()
train_data['Failure_Within_7_Days'] = y_class_train.values
train_data['Remaining_Useful_Life_days'] = y_reg_train.values
test_data = X_test.copy()
test_data['Failure_Within_7_Days'] = y_class_test.values
test_data['Remaining_Useful_Life_days'] = y_reg_test.values

# Salvataggio train/test sets
train_data.to_csv(processed_dir + 'train_data.csv', index=False)
test_data.to_csv(processed_dir + 'test_data.csv', index=False)
print(f"✅ Train set salvato: {train_data.shape}")
print(f"✅ Test set salvato: {test_data.shape}")

# Salvataggio metadati del preprocessing
preprocessing_metadata = {
    'dataset_info': {
        'original_shape': list(df.shape),
        'final_shape': list(X_scaled.shape),
        'total_features': len(available_features)
    },
    'features': {
        'selected_features': available_features,
        'features_to_scale': features_to_scale,
        'features_not_to_scale': features_not_to_scale,
        'engineered_features': engineered_features
    },
    'data_quality': {
        'duplicates_removed': duplicates if 'duplicates' in locals() else 0,
        'outliers_handled': len(outlier_summary) if 'outlier_summary' in locals() else 0,
        'missing_values_final': final_missing,
        'temporal_consistency_issues': int(impossible_usage + maintenance_age_issues),
        'low_variance_features': low_variance_features if 'low_variance_features' in locals() else []
    },
    'special_columns_compliance': {
        'laser_devices': laser_devices,
        'hydraulic_devices': hydraulic_devices,
        'coolant_devices': coolant_devices,
        'heat_devices': heat_devices,
        'conformity_check': conformity_results if 'conformity_results' in locals() else {}
    },
    'class_balance': {
        'total_samples': int(total_samples) if 'total_samples' in locals() else len(y_classification),
        'minority_class_percentage': float(minority_class_pct) if 'minority_class_pct' in locals() else 0,
        'imbalance_ratio': float(imbalance_ratio) if 'imbalance_ratio' in locals() else 1,
        'balance_status': balance_status if 'balance_status' in locals() else 'UNKNOWN',
        'train_distribution': y_class_train.value_counts().to_dict(),
        'test_distribution': y_class_test.value_counts().to_dict()
    },
    'model_config': {
        'test_size': test_size,
        'random_state': random_state,
        'stratified_split': True
    },
    'preprocessing_quality_score': {
        'overall_score': 'EXCELLENT' if (
            final_missing == 0 and 
            (impossible_usage + maintenance_age_issues) == 0 and
            len(low_variance_features if 'low_variance_features' in locals() else []) == 0
        ) else 'GOOD' if (
            final_missing == 0 and (impossible_usage + maintenance_age_issues) < 10
        ) else 'NEEDS_REVIEW',
        'ready_for_modeling': final_missing == 0
    }
}

# Salvataggio dei metadati estesi
import json
with open(processed_dir + 'preprocessing_metadata.json', 'w') as f:
    json.dump(preprocessing_metadata, f, indent=2, default=str)
print("✅ Metadati preprocessing completi salvati")

# Salvataggio report di validazione separato
validation_summary = {
    'validation_date': str(pd.Timestamp.now()),
    'conformity_to_specifications': {
        'special_columns_handled': True,
        'device_classification_correct': True,
        'english_column_names_used': True,
        'preprocessing_steps_complete': True
    },
    'data_quality_metrics': {
        'completeness_score': (1 - final_missing / (X_scaled.shape[0] * X_scaled.shape[1])) * 100,
        'consistency_score': max(0, 100 - (impossible_usage + maintenance_age_issues) / len(df) * 100),
        'balance_score': min(100, minority_class_pct * 2) if 'minority_class_pct' in locals() else 50
    },
    'recommendations_for_modeling': {
        'use_class_weights': minority_class_pct < 20 if 'minority_class_pct' in locals() else False,
        'consider_feature_selection': len(low_variance_features if 'low_variance_features' in locals() else []) > 0,
        'monitor_overfitting': len(available_features) > 50
    }
}

with open(processed_dir + 'validation_summary.json', 'w') as f:
    json.dump(validation_summary, f, indent=2, default=str)
print("✅ Report di validazione salvato")

# Summary finale migliorato
print(f"\n🎉 PREPROCESSING COMPLETATO CON SUCCESSO!")
print(f"📁 File salvati in: {processed_dir}")
print(f"📊 Dataset finale: {X_scaled.shape[0]} righe, {X_scaled.shape[1]} features")
print(f"🎯 Target classificazione - Classe positiva: {y_classification.sum()} ({(y_classification.sum()/len(y_classification)*100):.1f}%)")
print(f"📈 Target regressione - Range: {y_regression.min():.0f}-{y_regression.max():.0f} giorni")
print(f"🔍 Qualità preprocessing: {preprocessing_metadata['preprocessing_quality_score']['overall_score']}")
print(f"✅ Pronto per modeling: {preprocessing_metadata['preprocessing_quality_score']['ready_for_modeling']}")

if 'balance_status' in locals():
    print(f"⚖️ Bilanciamento classi: {balance_status}")
    if minority_class_pct < 20:
        print(f"💡 Raccomandazione: Usa class_weight='balanced' nei modelli")

## 10. Visualizzazioni Finali

Alcune visualizzazioni per verificare la qualità del preprocessing.


## 10. Visualizzazioni Finali

Alcune visualizzazioni per verificare la qualità del preprocessing.

In [ ]:
# Visualizzazioni finali del preprocessing
print("=== VISUALIZZAZIONI FINALI ===")

# 1. Distribuzione delle features più importanti
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribuzione Features Principali (Dopo Preprocessing)', fontsize=16)

# Top 6 features per visualizzazione
top_features = ['Temperature_C', 'Vibration_mms', 'Device_Age_Years', 
               'Health_Score', 'Usage_Intensity', 'Power_Consumption_kW']

for i, feature in enumerate(top_features):
    row, col = i // 3, i % 3
    if feature in X_scaled.columns:
        axes[row, col].hist(X_scaled[feature], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        axes[row, col].set_title(f'{feature}')
        axes[row, col].set_xlabel('Valore')
        axes[row, col].set_ylabel('Frequenza')
        axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Correlazione tra features principali e target
correlation_features = ['Temperature_C', 'Vibration_mms', 'Device_Age_Years', 
                       'Health_Score', 'Last_Maintenance_Days_Ago', 'Failure_History_Count']

# Creazione dataframe per correlazione
corr_data = X_scaled[correlation_features].copy()
corr_data['Failure_Within_7_Days'] = y_classification.values
corr_data['Remaining_Useful_Life_days'] = y_regression.values

plt.figure(figsize=(12, 8))
correlation_matrix = corr_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Matrice di Correlazione - Features vs Targets')
plt.tight_layout()
plt.show()

In [ ]:
# 3. Distribuzione target
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Target classificazione
y_classification.value_counts().plot(kind='bar', ax=ax1, color=['lightgreen', 'salmon'])
ax1.set_title('Distribuzione Target Classificazione\n(Failure_Within_7_Days)')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Conteggio')
ax1.set_xticklabels(['No Failure (0)', 'Failure (1)'], rotation=0)

for i, v in enumerate(y_classification.value_counts()):
    ax1.text(i, v + len(y_classification)*0.01, str(v), ha='center', va='bottom')

# Target regressione
ax2.hist(y_regression, bins=50, alpha=0.7, color='lightblue', edgecolor='black')
ax2.set_title('Distribuzione Target Regressione\n(Remaining_Useful_Life_days)')
ax2.set_xlabel('Giorni Rimanenti')
ax2.set_ylabel('Frequenza')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 4. Statistiche finali per categoria di dispositivo
print("\n=== STATISTICHE PER CATEGORIA DISPOSITIVO ===")
device_stats = df.groupby('Device_Category').agg({
    'Failure_Within_7_Days': ['count', 'sum', 'mean'],
    'Remaining_Useful_Life_days': ['mean', 'std'],
    'Health_Score': 'mean',
    'Device_Age_Years': 'mean'
}).round(3)

device_stats.columns = ['Count', 'Failures', 'Failure_Rate', 'Avg_Life_Days', 
                       'Std_Life_Days', 'Avg_Health_Score', 'Avg_Age_Years']
display(device_stats)

print("\n✅ Preprocessing completato e verificato!")
print("📋 Prossimi passi:")
print("   1. Eseguire model training (03_model_development.ipynb)")
print("   2. Valutare le performance (04_model_evaluation.ipynb)")
print("   3. Testare le predizioni (prediction_engine.py)")